## Εργασία 3 ##

Καλωσήρθατε στην τρίτη σας εργασία. Η εργασία αυτή έχει σκοπό να σας βοηθήσει να εμπεδώσετε τα σύνολα μοντέλων.

Στην εργασία αυτή θα πρέπει να συμπληρώσετε κώδικα Python 3 στα σημεία που αναφέρουν # YOUR CODE HERE. Μην τροποποιείτε τον κώδικα που βρίσκεται εκτός αυτών των περιοχών.

Πρωτού παραδόσετε την εργασία σας σιγουρευτείτε ότι ο κώδικας σε όλα τα κελιά τρέχει σωστά. Για το σκοπό αυτό επιλέξτε από το μενού Χρόνος εκτέλεσης (runtime) -> Επανεκίνηση περιόδου λειτουργίας και εκτέλεση όλων.

Συμπληρώστε το όνομα (NAME) και το AEM σας παρακάτω:

In [1]:
NAME = "DIMITRA GALANI"
AEM = "10331"

**1** Διαβάστε το διαθέσιμο από την sklearn σύνολο δεδομένων breast cancer, χωρίστε το σε δεδομένα εκπαίδευσης (X_train, y_train) και ελέγχου (X_test, y_test) σε ποσοστό 70%/30% αντίστοιχα με τη συνάρτηση train_test_split (τιμή για random_state βάλτε 0). Το σύνολο αφορά τη διάγνωση καρκίνου του μαστού με βάση μεταβλητές που υπολογίζονται από μια ψηφιοποιημένη εικόνα δείγματος μάζας μαστού που λήφθηκε μέσω αναρρόφησης λεπτής βελόνας (FNA). (2 μονάδες)

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Load the breast cancer dataset
breast_cancer = load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# Split the dataset into 70% training and 30% testing  sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
"""Τεστ ορθής ανάγνωσης και διαχωρισμού του συνόλου δεδομένων"""
assert round(X_train[0][8], 5) == 0.1779
assert round(X_test[0][8], 5) == 0.2116

**2** Υλοποιήστε μια ντετερμινιστική εκδοχή της μεθόδου των τυχαίων υποχώρων, η οποία χτίζει τόσα μοντέλα όσες και οι μεταβλητές εισόδου, κάθε ένα από τα οποία αγνοεί και μία διαφορετική μεταβλητή εισόδου. Π.χ. το πρώτο μοντέλο αγνοεί την πρώτη, το δεύτερο αγνοεί τη δεύτερη κτλ. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. (4 μονάδες)

In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class RandomSubspaceDet:
    def __init__(self, estimator=DecisionTreeClassifier()):
        self.base_estimator = estimator # The base estimator to train on
        self.models = [] # The trained models
        self.features_to_ignore = [] # The features to ignore for each model

    def fit(self, X_train, y_train):
        n_features = X_train.shape[1] # The number of features is the number of columns in X_train
        self.models = [] # Reset the models
        self.features_to_ignore = [] # Reset the features to ignore

        for i in range(n_features):
            # Create a copy of the base estimator at each iteration
            model = clone(self.base_estimator)
            # Store which feature to ignore
            self.features_to_ignore.append(i)
            X_train_subset = np.delete(X_train, i, axis=1) # Exclude the ith feature
            model.fit(X_train_subset, y_train)
            self.models.append(model)

    def predict(self, X):
        n_samples = X.shape[0]
        n_features = X.shape[1]

        # Initialize the predictions array
        predictions = np.zeros((n_samples, len(self.models)))

        # Collect predictions from each model
        for i, model in enumerate(self.models):
            # Predict using the data excluding the ith feature
            X_subset = np.delete(X, self.features_to_ignore[i], axis=1)
            predictions[:, i] = model.predict(X_subset)

        # Choose predictions by majority vote
        majority_votes = np.apply_along_axis(lambda x: np.bincount(x.astype(int)).argmax(), axis=1, arr=predictions)
        return majority_votes

In [5]:
"""Τεστ ορθής υλοποίησης RandomSubspaceDet"""
from sklearn.metrics import accuracy_score

rs = RandomSubspaceDet(estimator=DecisionTreeClassifier(random_state=1))
rs.fit(X_train, y_train)
assert round(accuracy_score(rs.predict(X_test), y_test), 4) == 0.9006

**3** Υλοποιήστε τη μέθοδο AdaBoost όπως παρουσιάστηκε στο μάθημα. Χρησιμοποιήστε τη συνάρτηση clone από το sklearn.base για να δημιουργήστε αντίγραφο του βασικού μοντέλου σε κάθε επανάληψη. Χρησιμοποιήστε την παράμετρο sample_weight της fit του βασικού μοντέλου για να ορίσετε τα βάρη των παραδειγμάτων εκπαίδευσης. (4 μονάδες)

In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone

class AdaBoost:
    def __init__(self, n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1)):
        self.n_estimators = n_estimators  # Number of estimators
        self.base_estimator = estimator  # Base estimator
        self.estimators = []  # List of trained estimators
        self.estimator_weights = []  # Weights of the estimators
        self.classes_ = None  # The number of classes 

    def fit(self, X_train, y_train):
        self.classes_ = len(np.unique(y_train))  # Store unique classes used in predict function
        n_samples = X_train.shape[0]
        sample_weights = np.ones(n_samples) / n_samples  # Initialize sample weights L2-L4 in pseudocode

        for t in range(self.n_estimators):
            estimator = clone(self.base_estimator)
            estimator.fit(X_train, y_train, sample_weight=sample_weights) #L6 in pseudocode
            predictions = estimator.predict(X_train)

            # Calculate the weighted error rate - L7 in pseudocode
            incorrect = (predictions != y_train)
            # print(sample_weights, incorrect)
            estimator_error = np.dot(sample_weights, incorrect)

            # If error is greater than 0.5 or 0, break - L8-L10 in pseudocode
            if estimator_error >= 0.5 or estimator_error == 0:
                break

            # Update sample weights based on correct and incorrect predictions - L11-L15 in pseudocode
            for i in range(n_samples):
                if predictions[i] == y_train[i]:
                    sample_weights[i] *= estimator_error / (1.0 - estimator_error)

            # Normalize sample weights - L16-L18 in pseudocode
            sample_weights /= np.sum(sample_weights)

            # Store the estimator and its weight
            self.estimators.append(estimator)
            # Compute the estimator's weight for the predict function - L22 in pseudocode
            estimator_weight = np.log((1.0 - estimator_error) / (estimator_error))
            self.estimator_weights.append(estimator_weight)

    
    def predict(self, X):
        n_samples = X.shape[0] # Number of samples
        n_classes = self.classes_ # Number of classes
        predictions = np.zeros((n_samples, n_classes)) # Initialize predictions array

        for i, estimator in enumerate(self.estimators):
            prediction = estimator.predict(X) # Get the predictions of the estimator
            # Update the predictions array with the weighted predictions
            for j in range(n_samples):
                predictions[j, prediction[j]] += self.estimator_weights[i] 
        return np.argmax(predictions, axis=1) # Return the class with the highest weight

    #! Accuracy score takes 2 arrays as input and compares them element-wise    
    

In [7]:
"""Τεστ ορθής υλοποίησης AdaBoost"""
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

ab = AdaBoost(n_estimators=20, estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591


In [8]:
# Ίδιο αποτέλεσμα και με τη κλάση της sklearn
ab = AdaBoostClassifier(n_estimators=20, algorithm="SAMME", estimator=DecisionTreeClassifier(max_depth=1, random_state=1))
ab.fit(X_train, y_train)
assert round(accuracy_score(ab.predict(X_test), y_test), 4) == 0.9591

TypeError: __init__() got an unexpected keyword argument 'estimator'